<a href="https://colab.research.google.com/github/Shaanu-cheeku/question-generation-project/blob/main/Code-Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## UPLOADING FILES

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WikiQA-dev.tsv to WikiQA-dev.tsv
Saving WikiQA-test.tsv to WikiQA-test.tsv
Saving WikiQA-train.tsv to WikiQA-train.tsv


##Tokenizing

In [ ]:
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# File paths (update as needed)
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names for the WikiQA data
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load the data into Pandas DataFrames
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to tokenize the dataset
def tokenize_data(data):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data)

# Prepare labels for each dataset
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# Define a custom Dataset class for WikiQA
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Verify DataLoader
for batch in train_loader:
    print("Sample Batch:")
    print(f"Input IDs: {batch['input_ids'].shape}")
    print(f"Attention Mask: {batch['attention_mask'].shape}")
    print(f"Labels: {batch['labels'].shape}")
    break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Tokenizing Dev Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Tokenizing Test Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Sample Batch:
Input IDs: torch.Size([16, 128])
Attention Mask: torch.Size([16, 128])
Labels: torch.Size([16])


##Fine tuning bert

In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
import torch

# Initialize the BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define training loop
epochs = 3
loss_fn = CrossEntropyLoss()

print("Starting training...")
for epoch in range(epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()  # Clear gradients

        # Move inputs and labels to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

print("Training completed!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting training...
Epoch 1/3, Loss: 0.1939


KeyboardInterrupt: 

## Checking Dataset

In [ ]:
import pandas as pd

# File paths (use raw strings to handle backslashes or replace them with forward slashes)
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names for the WikiQA data
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load the data into Pandas DataFrames
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# Preview the train data
print("Train Data Preview:")
print(train_data.head())

# Check the size of each dataset
print("\nDataset Sizes:")
print(f"Train: {len(train_data)} rows")
print(f"Dev: {len(dev_data)} rows")
print(f"Test: {len(test_data)} rows")


Train Data Preview:
                                 question_id      question document_title  \
Q1 how are glacier caves formed?          D1  Glacier cave           D1-0   
   how are glacier caves formed?          D1  Glacier cave           D1-1   
   how are glacier caves formed?          D1  Glacier cave           D1-2   
   how are glacier caves formed?          D1  Glacier cave           D1-3   
   how are glacier caves formed?          D1  Glacier cave           D1-4   

                                                                           sentence  \
Q1 how are glacier caves formed?  A partly submerged glacier cave on Perito More...   
   how are glacier caves formed?          The ice facade is approximately 60 m high   
   how are glacier caves formed?          Ice formations in the Titlis glacier cave   
   how are glacier caves formed?  A glacier cave is a cave formed within the ice...   
   how are glacier caves formed?  Glacier caves are often called ice caves , but..

##Tokenizing + Truncation

In [ ]:
import time
import pandas as pd
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to truncate inputs before tokenization
def truncate_inputs(row, max_tokens=100):
    question, sentence = row["question"], row["sentence"]
    combined_length = len(question.split()) + len(sentence.split())
    if combined_length > max_tokens:
        sentence = " ".join(sentence.split()[:max_tokens - len(question.split())])
    return question, sentence

# Apply truncation to train, dev, and test data
def truncate_data(data):
    data[["question", "sentence"]] = data.apply(
        lambda row: truncate_inputs(row), axis=1, result_type="expand"
    )
    return data

# Tokenize the dataset
def tokenize_data(data):
    start_time = time.time()
    encodings = tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    end_time = time.time()
    print(f"Tokenization completed in {end_time - start_time:.2f} seconds")
    return encodings

# File paths (update as needed)
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names for the WikiQA data
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load the data into Pandas DataFrames
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# Truncate inputs
print("Truncating Train Data...")
train_data = truncate_data(train_data)
print("Truncating Dev Data...")
dev_data = truncate_data(dev_data)
print("Truncating Test Data...")
test_data = truncate_data(test_data)

# Tokenize the datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data)

# Display an example tokenized output
print("Example tokenized train data (input IDs):", train_encodings["input_ids"][0])
print("Example tokenized train data (attention mask):", train_encodings["attention_mask"][0])


Truncating Train Data...
Truncating Dev Data...
Truncating Test Data...
Tokenizing Train Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Tokenization completed in 19.50 seconds
Tokenizing Dev Data...
Tokenization completed in 3.06 seconds
Tokenizing Test Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Tokenization completed in 5.47 seconds
Example tokenized train data (input IDs): tensor([  101, 10046,  5430,   102,  1037,  6576, 13563, 10046,  5430,  2006,
         2566,  9956, 17921, 10046,  1012,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

##Checking

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom Dataset class for WikiQA
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Prepare labels for each dataset
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders for batching
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Print a sample batch to verify
for batch in train_loader:
    print("Sample Batch:")
    print(f"Input IDs: {batch['input_ids'].shape}")
    print(f"Attention Mask: {batch['attention_mask'].shape}")
    print(f"Labels: {batch['labels'].shape}")
    break


Sample Batch:
Input IDs: torch.Size([16, 128])
Attention Mask: torch.Size([16, 128])
Labels: torch.Size([16])


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define a custom Dataset class for WikiQA
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Ensure tokenized data exists (train_encodings, train_labels, etc.)
# If not, revisit the tokenization step or rerun it

# Prepare tokenized data and labels for each dataset
train_labels = train_data["label"].tolist()  # Ensure train_data is loaded
train_dataset = WikiQADataset(train_encodings, train_labels)

# Create DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Print a sample batch to verify
for batch in train_loader:
    print("Sample Batch:")
    print(f"Input IDs: {batch['input_ids'].shape}")
    print(f"Attention Mask: {batch['attention_mask'].shape}")
    print(f"Labels: {batch['labels'].shape}")
    break


Sample Batch:
Input IDs: torch.Size([16, 82])
Attention Mask: torch.Size([16, 82])
Labels: torch.Size([16])


In [ ]:
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
import torch

# Initialize the BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define training loop
epochs = 3
loss_fn = CrossEntropyLoss()

print("Starting training...")
for epoch in range(epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()  # Clear gradients

        # Move inputs and labels to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

print("Training completed!")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'train_loader' is not defined

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(data, max_length=64):  # Reduced sequence length
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_data)
dev_encodings = tokenize_data(dev_data)
test_encodings = tokenize_data(test_data)

train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# Dataset
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

batch_size = 8  # Optimized batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Starting training...
Epoch 1/3, Loss: 0.1994


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation - Accuracy: 0.9488, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Epoch 2/3, Loss: 0.1673


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation - Accuracy: 0.9488, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Early stopping triggered!
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9524, Precision: 0.0000, Recall: 0.0000, F1: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9524198822759974, 0.0, 0.0, 0.0)

95%accuracy one

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Tokenization
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(data, max_length=64):  # Reduced sequence length
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 3: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 4: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 5: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 6: Evaluation Function
# =========================
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 7: Training Loop
# =========================
epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


Tokenizing Train Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Tokenizing Dev Data...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Tokenizing Test Data...


...

Evaluation - Accuracy: 0.9488, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Epoch 2/3, Loss: 0.7029


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation - Accuracy: 0.9488, Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Early stopping triggered!
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9524, Precision: 0.0000, Recall: 0.0000, F1: 0.0000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9524198822759974, 0.0, 0.0, 0.0)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets with dynamic max_length
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW  # Import AdamW optimizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets with dynamic max_length
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...
Starting training...
Epoch 1/3, Loss: 0.2281
Evaluation - Accuracy: 0.9378, Precision: 0.3295, Recall: 0.2071, F1: 0.2544
Epoch 2/3, Loss: 0.0365
Evaluation - Accuracy: 0.9396, Precision: 0.2727, Recall: 0.1071, F1: 0.1538
Epoch 3/3, Loss: 0.0186
Evaluation - Accuracy: 0.9444, Precision: 0.3500, Recall: 0.1000, F1: 0.1556
Early stopping triggered!
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9423, Precision: 0.1395, Recall: 0.0412, F1: 0.0637


(0.9422825376062786,
 0.13953488372093023,
 0.041237113402061855,
 0.0636604774535809)

--Increased Class Weights: Adjusted for the minority class.|Advanced Oversampling: Implemented.|Improved Threshold Tuning: Allows fine-grained control over predictions.|Support for More Epochs and Early Stopping: Enhanced training loop.--

More sophisticated techniques for handling imbalances.

Improved optimization strategies.

A focus on balanced metrics (F1, recall, precision) rather than just accuracy.

Additional evaluation and stopping mechanisms for efficiency.

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
gradient_accumulation_steps = 4  # Adjust to simulate larger batch sizes
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        with autocast():  # Mixed precision training
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            loss = loss / gradient_accumulation_steps  # Normalize loss
        scaler.scale(loss).backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...


<ipython-input-10-404c00cb0a65>:141: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-10-404c00cb0a65>:178: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Starting training...
Epoch 1/3, Loss: 0.1076
Evaluation - Accuracy: 0.8149, Precision: 0.1685, Recall: 0.6643, F1: 0.2688


<ipython-input-10-404c00cb0a65>:178: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Epoch 2/3, Loss: 0.0473
Evaluation - Accuracy: 0.8910, Precision: 0.2401, Recall: 0.5214, F1: 0.3288


<ipython-input-10-404c00cb0a65>:178: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Epoch 3/3, Loss: 0.0239
Evaluation - Accuracy: 0.9140, Precision: 0.2637, Recall: 0.3786, F1: 0.3109
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9040, Precision: 0.1992, Recall: 0.3368, F1: 0.2503


(0.9040222367560498,
 0.1991869918699187,
 0.33676975945017185,
 0.2503192848020434)

This updated code below integrates focal loss and threshold-based evaluation seamlessly into your existing setup

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Focal Loss Implementation
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, labels):
        probas = torch.nn.functional.softmax(logits, dim=1)
        labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=logits.size(-1)).float()
        focal_loss = -self.alpha * ((1 - probas) ** self.gamma) * labels_one_hot * torch.log(probas)
        focal_loss = focal_loss.sum(dim=1)
        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

loss_fn = FocalLoss(alpha=1.0, gamma=2.0).to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate_with_threshold(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
gradient_accumulation_steps = 4  # Adjust to simulate larger batch sizes
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        with autocast():  # Mixed precision training
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            loss = loss / gradient_accumulation_steps  # Normalize loss
        scaler.scale(loss).backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate_with_threshold(model, dev_loader, threshold=0.5)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate_with_threshold(model, test_loader, threshold=0.5)


Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...


<ipython-input-11-bf445a3aca64>:156: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-11-bf445a3aca64>:195: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Starting training...
Epoch 1/3, Loss: 0.0284
Evaluation - Accuracy: 0.7362, Precision: 0.1280, Recall: 0.7143, F1: 0.2172


<ipython-input-11-bf445a3aca64>:195: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Epoch 2/3, Loss: 0.0127
Evaluation - Accuracy: 0.8767, Precision: 0.2111, Recall: 0.5143, F1: 0.2994


<ipython-input-11-bf445a3aca64>:195: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision training


Epoch 3/3, Loss: 0.0068
Evaluation - Accuracy: 0.9140, Precision: 0.2705, Recall: 0.4000, F1: 0.3228
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9022, Precision: 0.1948, Recall: 0.3368, F1: 0.2469


(0.9022236756049705,
 0.19483101391650098,
 0.33676975945017185,
 0.24685138539042822)

Here’s the complete updated code with the new torch.amp.GradScaler() and torch.amp.autocast() syntax integrated:

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging
from torch.amp import GradScaler, autocast  # For mixed precision training

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance (Optional)
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])  # Shuffle the data
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Initialize GradScaler for mixed precision
scaler = torch.amp.GradScaler()

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate_with_threshold(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
gradient_accumulation_steps = 4  # Adjust to simulate larger batch sizes
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        with torch.amp.autocast(device_type="cuda"):  # Updated autocast for mixed precision
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            loss = loss / gradient_accumulation_steps  # Normalize loss
        scaler.scale(loss).backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate_with_threshold(model, dev_loader, threshold=0.5)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate_with_threshold(model, test_loader, threshold=0.5)


Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...
Starting training...
Epoch 1/3, Loss: 0.1076
Evaluation - Accuracy: 0.8149, Precision: 0.1685, Recall: 0.6643, F1: 0.2688
Epoch 2/3, Loss: 0.0473
Evaluation - Accuracy: 0.8910, Precision: 0.2401, Recall: 0.5214, F1: 0.3288
Epoch 3/3, Loss: 0.0239
Evaluation - Accuracy: 0.9140, Precision: 0.2637, Recall: 0.3786, F1: 0.3109
Training completed!
Testing on Test Data...
Evaluation - Accuracy: 0.9040, Precision: 0.1992, Recall: 0.3368, F1: 0.2503


(0.9040222367560498,
 0.1991869918699187,
 0.33676975945017185,
 0.2503192848020434)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils import resample
from transformers import logging
from torch.amp import GradScaler, autocast  # For mixed precision training

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

columns = ["question_id", "question", "document_title", "sentence", "label"]

train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
model_name = "distilbert-base-uncased"  # Use a smaller model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)
print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)
print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: Oversampling for Class Imbalance
# =========================
data = list(zip(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels))
data_positive = [d for d in data if d[2] == 1]
data_negative = [d for d in data if d[2] == 0]
data_positive_oversampled = resample(data_positive, replace=True, n_samples=len(data_negative), random_state=42)
balanced_data = data_positive_oversampled + data_negative
torch.random.manual_seed(42)
torch.utils.data.random_split(balanced_data, lengths=[len(balanced_data), 0])
input_ids, attention_masks, labels = zip(*balanced_data)
train_encodings = {
    "input_ids": torch.stack(input_ids),
    "attention_mask": torch.stack(attention_masks),
}
train_labels = list(labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

batch_size = 32  # Increase batch size for efficiency
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjusted learning rate
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

scaler = GradScaler()

# =========================
# Step 7: Evaluation Function
# =========================
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Evaluation - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return accuracy, precision, recall, f1

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        with autocast(device_type="cuda"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
    _, _, _, f1 = evaluate(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate(model, test_loader)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Starting training...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from transformers import logging
from torch.amp import GradScaler, autocast  # For mixed precision training

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_overflowing_tokens=False
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: SMOTE Oversampling
# =========================
print("Applying SMOTE for oversampling...")
smote = SMOTE(random_state=42)
input_ids_flat = train_encodings["input_ids"].numpy().reshape(len(train_labels), -1)
train_encodings_resampled, train_labels_resampled = smote.fit_resample(input_ids_flat, train_labels)

# Reshape resampled input IDs to tensor
train_encodings_resampled = {
    "input_ids": torch.tensor(train_encodings_resampled),
    "attention_mask": train_encodings["attention_mask"].repeat(len(train_labels_resampled) // len(train_labels), 1)
}

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings_resampled, train_labels_resampled)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Weighted loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

# =========================
# Step 7: Evaluation and Error Analysis
# =========================
def evaluate_with_error_analysis(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print(classification_report(all_labels, all_preds))
    return precision_recall_fscore_support(all_labels, all_preds, average="binary")

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
gradient_accumulation_steps = 4
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        with autocast():
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()
        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")
    _, _, f1, _ = evaluate_with_error_analysis(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate_with_error_analysis(model, test_loader)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...
Applying SMOTE for oversampling...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Starting training...


IndexError: index 31307 is out of bounds for dimension 0 with size 20347

MOVED TO CPU--ADJUSTED CODE ACCORDINGLY

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE
from transformers import logging
from torch.amp import autocast, GradScaler

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: SMOTE Oversampling
# =========================
print("Applying SMOTE for oversampling...")
smote = SMOTE(random_state=42)

# Flatten input_ids and attention_masks for SMOTE
input_ids_flat = train_encodings["input_ids"].numpy().reshape(len(train_labels), -1)
attention_masks_flat = train_encodings["attention_mask"].numpy().reshape(len(train_labels), -1)

# Apply SMOTE
input_ids_resampled, train_labels_resampled = smote.fit_resample(input_ids_flat, train_labels)

# Regenerate attention masks for resampled data
attention_masks_resampled = torch.tensor([
    [1] * dynamic_max_length for _ in range(len(train_labels_resampled))
])

# Update train_encodings with resampled data
train_encodings_resampled = {
    "input_ids": torch.tensor(input_ids_resampled),
    "attention_mask": attention_masks_resampled
}

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings_resampled, train_labels_resampled)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 7: Training Loop
# =========================
epochs = 3
print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

print("Training completed!")


Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...
Applying SMOTE for oversampling...
Starting training...


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from imblearn.over_sampling import SMOTE
from transformers import logging

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)

# =========================
# Step 2: Determine Dynamic max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def compute_dynamic_max_length(data, percentile=99):
    lengths = [
        len(tokenizer.encode(q, s, truncation=False)) for q, s in zip(data["question"], data["sentence"])
    ]
    max_length = sorted(lengths)[int(len(lengths) * (percentile / 100))]
    print(f"Dynamic max_length set to: {max_length}")
    return max_length

# Compute max_length dynamically based on the 99th percentile
dynamic_max_length = compute_dynamic_max_length(train_data)

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=dynamic_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=dynamic_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=dynamic_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

# =========================
# Step 4: SMOTE Oversampling
# =========================
print("Applying SMOTE for oversampling...")
smote = SMOTE(random_state=42)

# Convert input_ids to a 2D array for SMOTE
input_ids_flat = train_encodings["input_ids"].numpy().reshape(len(train_labels), -1)
resampled_input_ids, resampled_labels = smote.fit_resample(input_ids_flat, train_labels)

# Reconstruct attention_mask to match resampled input_ids
attention_mask_resampled = torch.ones_like(torch.tensor(resampled_input_ids))

# Reshape back to tensors
train_encodings_resampled = {
    "input_ids": torch.tensor(resampled_input_ids),
    "attention_mask": attention_mask_resampled
}
train_labels_resampled = torch.tensor(resampled_labels)

# =========================
# Step 5: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings_resampled, train_labels_resampled)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# =========================
# Step 6: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cpu")  # Explicitly set to CPU
model.to(device)

# Weighted loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 7: Evaluation and Error Analysis
# =========================
def evaluate_with_error_analysis(model, loader, threshold=0.5):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print(classification_report(all_labels, all_preds))
    return precision_recall_fscore_support(all_labels, all_preds, average="binary")

# =========================
# Step 8: Training Loop
# =========================
epochs = 3
early_stopping_patience = 2
best_f1 = 0.0
patience_counter = 0

print("Starting training...")
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")
    _, _, f1, _ = evaluate_with_error_analysis(model, dev_loader)
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break

print("Training completed!")
print("Testing on Test Data...")
evaluate_with_error_analysis(model, test_loader)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Dynamic max_length set to: 82
Tokenizing Train Data...
Tokenizing Dev Data...
Tokenizing Test Data...
Applying SMOTE for oversampling...


<ipython-input-3-7a84d19c2a6c>:99: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Starting training...


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import classification_report, precision_recall_fscore_support
from transformers import logging
from torch.nn import CrossEntropyLoss

# Suppress warnings from transformers
logging.set_verbosity_error()

# Enable CPU optimizations
torch.backends.mkldnn.enabled = True

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)
print("Data loaded successfully!")

# =========================
# Step 2: Set Static max_length
# =========================
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
static_max_length = 32  # Reduced sequence length for faster processing

# =========================
# Step 3: Tokenization
# =========================
def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize datasets
print("Tokenizing Train Data...")
train_encodings = tokenize_data(train_data, max_length=static_max_length)

print("Tokenizing Dev Data...")
dev_encodings = tokenize_data(dev_data, max_length=static_max_length)

print("Tokenizing Test Data...")
test_encodings = tokenize_data(test_data, max_length=static_max_length)

# Prepare labels
train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()

print("Tokenization completed!")

# =========================
# Step 4: Custom Dataset and DataLoader
# =========================
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

# Create PyTorch datasets
train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

# Create DataLoaders
batch_size = 32  # Increased batch size for faster processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print("DataLoaders created successfully!")

# =========================
# Step 5: Model and Loss Initialization
# =========================
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Calculate class weights
positive_count = sum(train_labels)
negative_count = len(train_labels) - positive_count
total_count = len(train_labels)
class_weights = torch.tensor([1.0, total_count / (2.0 * positive_count)]).to(device)

# Use CrossEntropyLoss with class weights
loss_fn = CrossEntropyLoss(weight=class_weights)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
num_training_steps = len(train_loader) * 5  # For 5 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

print("Model loaded on", device)

# =========================
# Step 6: Training Loop with Debugging
# =========================
epochs = 5
best_f1 = 0.0
patience = 2  # Early stopping patience
no_improvement_epochs = 0
decision_threshold = 0.5

print("Starting training...")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()

        # Track accuracy during training
        preds = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_samples += labels.size(0)

    train_accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Evaluate on dev set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dev_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > decision_threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Dev Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Save the best model
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_model.pt")
        print(f"Checkpoint saved at epoch {epoch + 1}")
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
        if no_improvement_epochs >= patience:
            print("Early stopping triggered!")
            break

print("Training completed!")

# =========================
# Step 7: Final Evaluation
# =========================
print("Evaluating on Test Data...")
model.load_state_dict(torch.load("best_model.pt"))
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
        preds = (probas > decision_threshold).long()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print("Test Set Classification Report:")
print(classification_report(all_labels, all_preds))


Data loaded successfully!


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dev Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Epoch 2/5, Loss: 395.5525, Training Accuracy: 0.9433


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-2-35b194b5bb10>:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

Dev Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Early stopping triggered!
Training completed!
Evaluating on Test Data...


FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pt'

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import classification_report, precision_recall_fscore_support
from transformers import logging
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Suppress warnings from transformers
logging.set_verbosity_error()

# =========================
# Step 1: Data Loading
# =========================
# File paths
train_file = "/content/WikiQA-train.tsv"
dev_file = "/content/WikiQA-dev.tsv"
test_file = "/content/WikiQA-test.tsv"

# Define column names
columns = ["question_id", "question", "document_title", "sentence", "label"]

# Load data
print("Loading data...")
train_data = pd.read_csv(train_file, sep="\t", names=columns, header=0)
dev_data = pd.read_csv(dev_file, sep="\t", names=columns, header=0)
test_data = pd.read_csv(test_file, sep="\t", names=columns, header=0)
print("Data loaded successfully!")

# =========================
# Step 2: Tokenization
# =========================
print("Tokenizing data...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
static_max_length = 64  # Adjust for faster processing

def tokenize_data(data, max_length):
    return tokenizer(
        data["question"].tolist(),
        data["sentence"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_data, static_max_length)
dev_encodings = tokenize_data(dev_data, static_max_length)
test_encodings = tokenize_data(test_data, static_max_length)

train_labels = train_data["label"].tolist()
dev_labels = dev_data["label"].tolist()
test_labels = test_data["label"].tolist()
print("Tokenization completed!")

# =========================
# Step 3: Dataset and DataLoader
# =========================
print("Creating DataLoaders...")
class WikiQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

train_dataset = WikiQADataset(train_encodings, train_labels)
dev_dataset = WikiQADataset(dev_encodings, dev_labels)
test_dataset = WikiQADataset(test_encodings, test_labels)

batch_size = 16  # Adjust batch size for better runtime
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
print("DataLoaders created successfully!")

# =========================
# Step 4: Model and Optimizer Initialization
# =========================
print("Loading model...")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on {device}")

# Calculate class weights
class_weights = compute_class_weight(
    "balanced",
    classes=np.array([0, 1]),  # Convert to NumPy array
    y=np.array(train_labels)  # Convert labels to NumPy array
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = len(train_loader) * 5
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# =========================
# Step 5: Training Loop
# =========================
epochs = 5
best_f1 = 0.0
patience = 3
no_improvement_epochs = 0
best_threshold = 0.3

print("Starting training...")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()

        preds = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_samples += labels.size(0)

    train_accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Evaluate on dev set
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dev_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > best_threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="binary")
    print(f"Dev Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Save the best model
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_model.pt")
        print(f"Checkpoint saved at epoch {epoch + 1}")
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1
        if no_improvement_epochs >= patience:
            print("Early stopping triggered!")
            break

print("Training completed!")

# =========================
# Step 6: Final Evaluation
# =========================
print("Evaluating on Test Data...")
try:
    model.load_state_dict(torch.load("best_model.pt"))
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probas = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = (probas > best_threshold).long()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print("Test Set Classification Report:")
    print(classification_report(all_labels, all_preds))
except FileNotFoundError:
    print("No best model checkpoint found. Please check the training process.")


Loading data...
Data loaded successfully!
Tokenizing data...
Tokenization completed!
Creating DataLoaders...
DataLoaders created successfully!
Loading model...
Model loaded on cpu
Starting training...
Epoch 1/5, Loss: 700.7712, Training Accuracy: 0.8672
Dev Precision: 0.1562, Recall: 0.6429, F1: 0.2514
Checkpoint saved at epoch 1
